# Import Necessary Libraries

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import shap
import torch
from Explainer import *

# Load Dataset

In [2]:
data = pd.read_csv("Adult/Dataset/adult.csv")

# Explore Dataset

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

# Data Preprocessing

In [3]:
data = data.replace('?', np.nan)

In [4]:
# Drop Missing Values
data.dropna(how='any', inplace=True)

In [5]:
data.drop_duplicates(inplace=True)

In [6]:
data['income']= data['income'].replace({'<=50K':0, '>50K':1})

In [ ]:
data.head()

In [7]:
# Only 2 subrgoups in Race, White and Other 
# Other subgroup represents Black, Asian-Pac-Islander, Amer-Indian-Eskimo and Other
data['race'] = data['race'].apply(lambda x: 0 if x == 'White' else 1)

In [ ]:
race_counts = data['race'].value_counts()
print("Race distribution:\n", race_counts)
sns.countplot(x='race', data=data)
plt.title('Race Distribution')
plt.show()

In [ ]:
gender_counts = data['gender'].value_counts()
print("Gender distribution:\n", gender_counts)
sns.countplot(x='gender', data=data)
plt.title('Gender Distribution')
plt.show()

In [8]:
# Encode Gender, Male = 0 / Female = 1
data['gender'] = data['gender'].apply(lambda x: 0 if x == 'Male' else 1)
data.rename(columns={'gender': 'sex'}, inplace=True)

In [9]:
label_encoder = LabelEncoder()
data['workclass'] = label_encoder.fit_transform(data['workclass'])
data['education'] = label_encoder.fit_transform(data['education'])
data['marital-status'] = label_encoder.fit_transform(data['marital-status'])
data['occupation'] = label_encoder.fit_transform(data['occupation'])
data['relationship'] = label_encoder.fit_transform(data['relationship'])
data['native-country'] = label_encoder.fit_transform(data['native-country'])

In [9]:
data['capital-gain'] = (data['capital-gain']/(data['capital-gain'].max() - data['capital-gain'].min()))*(999-0)

In [ ]:
# Group the data by gender and outcome
grouped = data.groupby(['race', 'income']).size().unstack(fill_value=0)

# Create a bar plot
grouped.plot(kind='bar', figsize=(8, 6))

# Set plot labels and title
plt.title('Outcome Distribution by Gender')
plt.xlabel('sex')
plt.ylabel('Count')
plt.xticks(rotation=0) # Rotate x-axis labels for readability
plt.legend(title='Income', labels=['Income <= 50K', 'Income > 50K'])

# Split the data into Training and Testing

In [ ]:
# # Take only 14680 individual of each group
# males = data[data['race'] == 0].sample(n=6316, random_state=42)
# females = data[data['race'] == 1].sample(n=6316, random_state=42)
# filtered_data = pd.concat([males, females])
# data = filtered_data.sample(frac=1, random_state=42)

In [10]:
X = data.drop('income', axis=1)
y = data['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40, stratify=y)

# Balance The Dataset

#### Over Sampling

In [11]:
# Apply SMOTE (oversampling) only to the training data to balance the minority class
smote = SMOTE(random_state=42, sampling_strategy='auto')
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [12]:
# Combine the gender and race columns back into the balanced dataset
df_balanced = pd.concat([X_train_balanced, y_train_balanced], axis=1)

#### Under Sampling

In [ ]:
# Apply RandomUnderSampler (undersampling) to ensure majority class isn't too dominant
undersampler = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = undersampler.fit_resample(X_train, y_train)

In [ ]:
# Combine the gender and race columns back into the balanced dataset
df_balanced = pd.concat([X_train_balanced, y_train_balanced], axis=1)

In [ ]:
# Group the data by gender and outcome
grouped = df_balanced.groupby(['race', 'income']).size().unstack(fill_value=0)

# Create a bar plot
grouped.plot(kind='bar', figsize=(8, 6))

# Set plot labels and title
plt.title('Outcome Distribution by Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=0) # Rotate x-axis labels for readability
plt.legend(title='Income', labels=['Income < 50K', 'Income >= 50K'])

# Random Forest Classifier

In [22]:
naiveModel = RandomForestClassifier(max_depth=15, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=200) #max_depth=15, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=200
naiveModel.fit(X_train_balanced,y_train_balanced) #X_train_balanced,y_train_balanced
y_pred = naiveModel.predict(X_test)

In [13]:
filename = 'random_forest_classifier.sav'

In [14]:
naiveModel = pickle.load(open('Adult/Models/'+filename, 'rb'))
y_pred = naiveModel.predict(X_test)

# Gaussian Naive Bayes Model

In [ ]:
naiveModel = GaussianNB()
naiveModel.fit(X_train,y_train) #X_train_balanced,y_train_balanced
y_pred = naiveModel.predict(X_test)

In [ ]:
filename = 'gaussian_naive_bayes.sav'

# Prediction Report

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
confusion = confusion_matrix(y_test, y_pred)
print(confusion)
sns.heatmap(confusion, annot=True, fmt=".2f")

# Save the Model

In [22]:
#Save the trained model
pickle.dump(naiveModel, open('Adult/Models/'+filename, 'wb'))

# Test Model Fairness

In [15]:
# fairlearn
from fairlearn.metrics import (
    false_positive_rate,
    false_negative_rate,
    true_positive_rate,
    MetricFrame,
    equalized_odds_difference,
    demographic_parity_difference,
)

def score(
    y_train,
    y_train_pred,
    y_test,
    y_test_pred,
    sensitive_features_train,
    sensitive_features_test,
    metrics={"accuracy": accuracy_score, "fpr": false_positive_rate, "fnr": false_negative_rate,},
):
    """
    Helper function to evaluate classifiers without too much repetition of code.
    """

    # training set
    mf_train = MetricFrame(
        metrics=metrics,
        y_true=y_train,
        y_pred=y_train_pred,
        sensitive_features=sensitive_features_train,
    )

    # test set
    mf_test = MetricFrame(
        metrics=metrics,
        y_true=y_test,
        y_pred=y_test_pred,
        sensitive_features=sensitive_features_test,
    )

    # display results
    display(
        pd.concat(
            [mf_train.by_group, mf_test.by_group], keys=["train", "test"]
        ).unstack(level=0)
    )

    # compute metrics
    print(
        "equalized odds (test): %.2f"
        % equalized_odds_difference(
            y_true=y_test,
            y_pred=y_test_pred,
            sensitive_features=sensitive_features_test,
        )
    )

    print("accuracy (test): %.2f" % accuracy_score(y_true=y_test, y_pred=y_test_pred))
    return

In [ ]:
score(
    y_train_balanced,
    naiveModel.predict(X_train_balanced),
    y_test,
    naiveModel.predict(X_test),
    X_train_balanced["race"],
    X_test["race"],
)

# UnFairness Mitigation

#### In Process Using Exponentiated Gradient

In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds, TruePositiveRateParity

constraint = EqualizedOdds()
rf = RandomForestClassifier()

fair_model = ExponentiatedGradient(rf, constraints=constraint)
fair_model.fit(X_train_balanced, y_train_balanced, sensitive_features=X_train_balanced[['race']])
y_pred_fair = fair_model.predict(X_test)

#### Grid Search

In [ ]:
from fairlearn.reductions import GridSearch, DemographicParity, EqualizedOdds
from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference

# Define your base estimator
base_estimator = RandomForestClassifier()

# Apply GridSearch with a fairness constraint
mitigator = GridSearch(estimator=base_estimator,
                       constraints=DemographicParity(),
                       grid_size=10)

# Fit the mitigator
mitigator.fit(X_train_balanced, y_train_balanced, sensitive_features=X_train_balanced[['sex']])

# Access the models generated by GridSearch
models = mitigator.predictors_

# Evaluate and select the best model based on accuracy and fairness trade-off
fair_model = None
best_score = float('-inf')

for model in models:
    y_pred = model.predict(X_test)
    
    # Calculate accuracy and fairness metrics
    accuracy = accuracy_score(y_test, y_pred)
    dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test[['sex']])
    eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=X_test[['sex']])
    
    # Define your custom criteria to select the best model
    score = accuracy - (dp_diff + eo_diff) # Example: accuracy minus DP difference
    
    if score > best_score:
        best_score = score
        fair_model = model

# Use best_model for predictions
y_pred_fair = fair_model.predict(X_test)

In [ ]:
dp_diff_final = demographic_parity_difference(y_test, y_pred_fair, sensitive_features=X_test[['race']])
eo_diff_final = equalized_odds_difference(y_test, y_pred_fair, sensitive_features=X_test[['race']])

print(classification_report(y_test, y_pred_fair))
confusion = confusion_matrix(y_test, y_pred_fair)
print(confusion)
sns.heatmap(confusion, annot=True, fmt=".2f")
print(accuracy_score(y_test,y_pred_fair))
print(f"Final Demographic Parity Difference: {dp_diff_final}")
print(f"Final Equalized Odds Difference: {eo_diff_final}")

# Test Fairness of New Fair Model

In [ ]:
from raiwidgets import FairnessDashboard

sensitive_features = X_test[['race', 'sex']]
FairnessDashboard(
    sensitive_features=sensitive_features,
    y_true=y_test,
    y_pred=y_pred_fair
)

In [ ]:
# Find instances where the model predicted target class 0
predicted_target_0_instances = X_test[y_pred == 0]
print(predicted_target_0_instances['race'].value_counts())

# males = predicted_target_0_instances[predicted_target_0_instances['sex'] == 0].sample(n=961, random_state=42)  # 42 is a random seed
# females = predicted_target_0_instances[predicted_target_0_instances['sex'] == 1].sample(n=961, random_state=42)
# filtered_test_data = pd.concat([males, females])
# predicted_target_0_instances = filtered_test_data.sample(frac=1, random_state=42)

# Save the filtered instances to a new dataset
predicted_target_0_instances.to_csv('Adult/Dataset/predicted_target_0_instances.csv', index=False)

# Test Which Features are Influencing the Prediction

In [ ]:
shap_explainer = shap.Explainer(naiveModel)
shap_values = shap_explainer.shap_values(X_train_balanced)
shap.summary_plot(shap_values, X_train_balanced, feature_names=data.columns)

# RL Agent Training

In [ ]:
dataset = data
affected_dataset = predicted_target_0_instances
model = naiveModel #fair_model   #naiveModel
target = 1
protected_attribute = "race"
features_to_change = ["capital-gain", "hours-per-week"]
minimums = [0, 1]
maximums = [99999, 99]
number_of_counterfactuals = 5

explainer = Explainer(dataset, affected_dataset, model, protected_attribute, features_to_change, number_of_counterfactuals, target, minimums, maximums, action_effectiveness=0.7)
explainer.train()

x, y = explainer.plot()
plt.show()
plt.plot(x, y)
plt.xlabel("Number of Timesteps")
plt.ylabel("Rewards")
plt.title("Learning Curve" + " Smoothed")
plt.show()

cfs = explainer.report_counterfactuals()
print(cfs)